# TextAttack End-to-End

This tutorial provides a broad end-to-end overview of training, evaluating, and attacking a model using TextAttack.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/0_End_to_End.ipynb)

## Training

First, we're going to train a model. TextAttack integrates directly with [transformers](https://github.com/huggingface/transformers/) and [datasets](https://github.com/huggingface/datasets) to train any of the `transformers` pre-trained models on datasets from `datasets`. 

Let's use the SNLI textual entailment dataset: it's relatively short (in word count, at least), and showcases a lot of the features of `textattack train`. Let's take a look at the dataset using `textattack peek-dataset`:

In [1]:
!textattack peek-dataset --dataset-from-huggingface snli

textattack: Loading nlp dataset snli, split train.
textattack: Number of samples: 550152
textattack: Number of words per input:
textattack: 	total:   11194676
textattack: 	mean:    20.35
textattack: 	std:     7.01
textattack: 	min:     4
textattack: 	max:     114
textattack: Dataset lowercased: False
textattack: First sample:
Premise: A person on a horse jumps over a broken down airplane.
Hypothesis: A person is training his horse for a competition. 

textattack: Last sample:
Premise: A man is surfing in a bodysuit in beautiful blue water.
Hypothesis: On the beautiful blue water there is a man in a bodysuit surfing. 

textattack: Found 4 distinct outputs.
textattack: Most common outputs:
	 0      (183416)
	 2      (183187)
	 1      (182764)
	 -1     (785)


The dataset looks good! It's not lowercased already, so we'll make sure our model is cased. Looks like there are some missing (-1) labels, so we need to filter those out. The longest input is 114 words, so we can cap our maximum sequence length (`--max-length`) at 128.

We'll train [`distilbert-base-cased`](https://huggingface.co/transformers/model_doc/distilbert.html), since it's a relatively small model, and a good example of how we integrate with `transformers`.

So we have our command:

```bash
textattack train                  \ # Train a model with TextAttack
    --model distilbert-base-cased \ # Using distilbert, cased version, from `transformers`
    --dataset snli                \ # On the SNLI dataset
    --max-length 128              \ # With a maximum sequence length of 128
    --batch-size 256              \ # And a batch size of 256
    --epochs 3                    \ # For 3 epochs
    --allowed-labels 0 1 2          # And only allow labels 0, 1, 2 (filter out -1!)
```

Now let's run it:

In [8]:
!textattack train --model distilbert-base-cased --dataset snli --max-length 128 --batch-size 256 --epochs 3 --allowed-labels 0 1 2

textattack: WARNING: TextAttack's model training feature is in beta. Please report any issues on our Github page, https://github.com/QData/TextAttack/issues.
textattack: Writing logs to /textattack/outputs/training/distilbert-base-cased-snli-2020-06-24-14:03/log.txt.
textattack: Loading nlp dataset snli, split train.
textattack: Loading nlp dataset snli, split validation.
textattack: Filtering samples with labels outside of [0, 1, 2].
textattack: Filtered 550152 train samples to 549367 points.
textattack: Filtered 10000 dev samples to 9842 points.
textattack: Loaded dataset. Found: 3 labels: ([0, 1, 2])
textattack: Loading transformers AutoModelForSequenceClassification: distilbert-base-cased
AutoTokenizer using fast tokenizer
textattack: Tokenizing training data. (len: 549367)
textattack: Tokenizing eval data (len: 9842)
textattack: Loaded data and tokenized in 102.62434101104736s
textattack: Training model across 4 GPUs
textattack: ***** Running training *****
textattack: 	Num exampl

## Evaluation

We successfully fine-tuned `distilbert-base-cased` for 3 epochs. Now let's evaluate it using `textattack eval`. This is as simple as providing the path to the pretrained model to `--model`, along with the number of evaluation samples. `textattack eval` will automatically load the evaluation data from training:

In [12]:
!textattack eval --num-examples 1000 --model /textattack/outputs/training/distilbert-base-cased-snli-2020-06-24-14:03/

textattack: Loading transformers AutoModelForSequenceClassification: /textattack/outputs/training/distilbert-base-cased-snli-2020-06-24-14:03/
textattack: Loading nlp dataset snli, split validation.
Successes 868/1000 (86.80%)


Awesome -- we were able to train a model up to 86.8% validation-set accuracy– with only a single command!

## Attack

Finally, let's attack our pre-trained model. We can do this the same way as before (by providing the path to the pretrained model to `--model`). For our attack, let's use the "TextFooler" attack recipe, from the paper ["Is BERT Really Robust? A Strong Baseline for Natural Language Attack on Text Classification and Entailment" (Jin et al, 2019)](https://arxiv.org/abs/1907.11932). We can do this by passing `--recipe textfooler` to `textattack attack`.

> *Warning*: We're printing out 1000 examples and, if the attack succeeds, their perturbations. The output of this command is going to be quite long!


In [1]:
!textattack attack --recipe textfooler --num-examples 1000 --model /textattack/outputs/training/distilbert-base-cased-snli-2020-06-24-14:03/

textattack: Loading transformers AutoModelForSequenceClassification: /textattack/outputs/training/distilbert-base-cased-snli-2020-06-24-14:03/
Using /u/jxmorris12/.cache/tensorflow-hub to cache modules.
textattack: Goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'> compatible with model DistilBertForSequenceClassification.
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  unk
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  50
    (embedding_type):  paragramcf
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding_type):  paragramcf
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angular
        (thr

[Succeeded / Failed / Total] 10 / 0 / 10:   1%| | 10/1000 [00:04<07:44,  2.13it/--------------------------------------------- Result 11 ---------------------------------------------
Contradiction (100%) --> Neutral (49%)

Premise: Two young boys of opposing teams play football, while wearing full protection uniforms and helmets.
Hypothesis: dog eats out of bowl

Premise: Two young fellers of opposing teams cheek football, while wearing full protection uniforms and helmets.
Hypothesis: whelp eaten out of bowl


[Succeeded / Failed / Total] 11 / 0 / 11:   1%| | 11/1000 [00:05<07:30,  2.19it/--------------------------------------------- Result 12 ---------------------------------------------
Neutral (87%) --> Contradiction (56%)

Premise: Two young boys of opposing teams play football, while wearing full protection uniforms and helmets.
Hypothesis: boys scoring a touchdown

Premise: Two young boys of opposing teams play football, while wearing full protection uniforms and helmets.
Hypothe

[Succeeded / Failed / Total] 22 / 1 / 24:   2%| | 24/1000 [00:08<05:34,  2.92it/--------------------------------------------- Result 25 ---------------------------------------------
Neutral (56%) --> Contradiction (77%)

Premise: A young boy in a field of flowers carrying a ball
Hypothesis: boy leaving baseball game

Premise: A young boy in a field of flowers carrying a ball
Hypothesis: muchachos leaving baseball game


[Succeeded / Failed / Total] 23 / 1 / 25:   2%| | 25/1000 [00:08<05:27,  2.98it/--------------------------------------------- Result 26 ---------------------------------------------
Entailment (69%) --> Contradiction (80%)

Premise: A young boy in a field of flowers carrying a ball
Hypothesis: boy in field

Premise: A young boy in a realms of flowers carrying a ball
Hypothesis: boy in field


[Succeeded / Failed / Total] 24 / 1 / 26:   3%| | 26/1000 [00:08<05:19,  3.05it/--------------------------------------------- Result 27 --------------------------------------------

[Succeeded / Failed / Total] 35 / 2 / 39:   4%| | 39/1000 [00:11<04:36,  3.48it/--------------------------------------------- Result 40 ---------------------------------------------
Contradiction (96%) --> Neutral (52%)

Premise: At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.
Hypothesis: A single man is next to a camera

Premise: At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.
Hypothesis: A unmarried fellers is next to a camera


[Succeeded / Failed / Total] 36 / 2 / 40:   4%| | 40/1000 [00:11<04:34,  3.50it/--------------------------------------------- Result 41 ---------------------------------------------
Neutral (56%) --> Entailment (37%)

Premise: At an outdoor event in an Asian-themed area, a crowd congregates as one person in a yellow Chinese dragon costume confronts the camera.
Hypothesis: A crowd is dan

[Succeeded / Failed / Total] 48 / 3 / 54:   5%| | 54/1000 [00:13<03:55,  4.02it/--------------------------------------------- Result 55 ---------------------------------------------
Neutral (90%) --> Contradiction (66%)

Premise: A older gentleman finds it humorous that he is getting his picture taken while doing his laundry.
Hypothesis: a blond man laughs while doing laundry

Premise: A older gentleman finds it humorous that he is getting his picture taken while doing his apparel.
Hypothesis: a blond man laughs while doing laundry


[Succeeded / Failed / Total] 49 / 3 / 55:   6%| | 55/1000 [00:13<03:53,  4.04it/--------------------------------------------- Result 56 ---------------------------------------------
Entailment (56%) --> Neutral (47%)

Premise: A older gentleman finds it humorous that he is getting his picture taken while doing his laundry.
Hypothesis: a guy laughs while doing laundry

Premise: A older gentleman finds it humorous that he is getting his picture taken while d

...

[Succeeded / Failed / Total] 714 / 96 / 931:  93%|▉| 931/1000 [03:09<00:14,  4.9--------------------------------------------- Result 932 ---------------------------------------------
Entailment (99%) --> Neutral (50%)

Premise: A man in yellow, white, and blue bicycle clothing and helmet grimaces as he rides his bicycle.
Hypothesis: A man is riding a bicycle.

Premise: A man in yellow, white, and blue bicycle clothing and helmet grimaces as he rides his bicycle.
Hypothesis: A man is riding a biking.


[Succeeded / Failed / Total] 715 / 96 / 932:  93%|▉| 932/1000 [03:09<00:13,  4.9--------------------------------------------- Result 933 ---------------------------------------------
Contradiction (100%) --> Neutral (50%)

Premise: A man in yellow, white, and blue bicycle clothing and helmet grimaces as he rides his bicycle.
Hypothesis: A girl is riding a roller coaster.

Premise: A man in yellow, white, and blue bicycle clothing and helmet grimaces as he rides his biking.
Hypothesis: ano

[Succeeded / Failed / Total] 728 / 96 / 946:  95%|▉| 946/1000 [03:11<00:10,  4.9--------------------------------------------- Result 947 ---------------------------------------------
Contradiction (100%) --> [FAILED]

Premise: A man in his own world of him and his music just entertaining on the street corner.
Hypothesis: A man is eating at a local diner.


[Succeeded / Failed / Total] 728 / 97 / 947:  95%|▉| 947/1000 [03:12<00:10,  4.9--------------------------------------------- Result 948 ---------------------------------------------
Neutral (97%) --> Contradiction (53%)

Premise: A man in his own world of him and his music just entertaining on the street corner.
Hypothesis: A man is enjoying playing music for money.

Premise: A man in his peculiar world of him and his songs just entertained on the rue corners.
Hypothesis: A fella is savoring playing music for money.


[Succeeded / Failed / Total] 729 / 97 / 948:  95%|▉| 948/1000 [03:12<00:10,  4.9------------------------------------

[Succeeded / Failed / Total] 739 / 98 / 960:  96%|▉| 960/1000 [03:15<00:08,  4.9--------------------------------------------- Result 961 ---------------------------------------------
Contradiction (99%) --> Neutral (55%)

Premise: A group of children playing and having a good time.
Hypothesis: The children are reading books.

Premise: A group of children jeu and having a good time.
Hypothesis: The children are reading books.


[Succeeded / Failed / Total] 740 / 98 / 961:  96%|▉| 961/1000 [03:15<00:07,  4.9--------------------------------------------- Result 962 ---------------------------------------------
Entailment (97%) --> Neutral (45%)

Premise: Inside the igloo, the young man gets ready for his overnight stay.
Hypothesis: The young man gets ready for his overnight stay.

Premise: Inside the igloo, the teenaged man gets ready for his overnight stay.
Hypothesis: The young man gets ready for his soiree stay.


[Succeeded / Failed / Total] 741 / 98 / 962:  96%|▉| 962/1000 [03:15<00:0

[Succeeded / Failed / Total] 751 / 98 / 975:  98%|▉| 975/1000 [03:17<00:05,  4.9--------------------------------------------- Result 976 ---------------------------------------------
Contradiction (91%) --> Neutral (57%)

Premise: Man in brown and red checkered button up shirt and blue jeans makes a male and female laugh.
Hypothesis: A flight attendant is demonstrating airplane safety but no one is paying attention.

Premise: Man in brown and red checkered button up shirt and blue jeans makes a male and female laugh.
Hypothesis: A flight attendant is illustrating airplane safety but no one is paying heed.


[Succeeded / Failed / Total] 752 / 98 / 976:  98%|▉| 976/1000 [03:17<00:04,  4.9--------------------------------------------- Result 977 ---------------------------------------------
Entailment (97%) --> Neutral (69%)

Premise: A girl with a purple sweater and a green headband is doing classwork in a classroom.
Hypothesis: A girl is doing work in a classroom.

Premise: A girl with a

[Succeeded / Failed / Total] 763 / 98 / 989:  99%|▉| 989/1000 [03:19<00:02,  4.9--------------------------------------------- Result 990 ---------------------------------------------
Neutral (94%) --> Contradiction (44%)

Premise: Three people are kayaking in dark waters.
Hypothesis: A couple and a guide are whitewater rafting.

Premise: Three people are kayaking in dark aqueduct.
Hypothesis: A paired and a directions are whitewater rafting.


[Succeeded / Failed / Total] 764 / 98 / 990:  99%|▉| 990/1000 [03:19<00:02,  4.9--------------------------------------------- Result 991 ---------------------------------------------
Contradiction (75%) --> [SKIPPED]

Premise: Three people are kayaking in dark waters.
Hypothesis: Three people are learning to use pogo sticks.


[Succeeded / Failed / Total] 764 / 98 / 991:  99%|▉| 991/1000 [03:19<00:01,  4.9--------------------------------------------- Result 992 ---------------------------------------------
Entailment (84%) --> Neutral (54%)

Prem

Looks like our model was 86.8% successful (makes sense - same evaluation set as `textattack eval`!), meaning that TextAttack attacked the model with 868 examples (since the attack won't run if an example is originally mispredicted). The attack success rate was 88.7%, meaning that TextFooler failed to find an adversarial example only 11.3% of the time.


## Conclusion

That's all, folks! We've learned how to train, evaluate, and attack a model with TextAttack, using only three commands! 😀